In [6]:
import pandas as pd
import numpy as np
from pprint import pprint
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MinMaxScaler

import nltk
# from nltk import word_tokenize - нужно nltk.download('punkt')

from nltk import wordpunct_tokenize, wordnet
from nltk.stem import wordnet as WordNetLem
from nltk.stem import SnowballStemmer, StemmerI

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import doc2vec
from gensim.models import LdaModel, LdaMulticore
from gensim.models import LsiModel
from gensim import models

In [7]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/Авалон/Диплом/')

In [9]:
stop_words = \
    ['и', 'в', 'во',  'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
     'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от',
     'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже',
     'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там',
     'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы',
     'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под',
     'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь',
     'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех',
     'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот',
     'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя',
     'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой',
     'им', 'более', 'всегда', 'конечно', 'всю', 'между',]#'не', ]
stop_words.extend(['очень', 'ооочень', 'это', 'данное'])

# Функции для обработки

In [1]:
# токенизация по словам (с удалением пунктуации)
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# Загрузка данных

In [10]:
df = pd.read_csv('data/coffee.csv')
df['rating'] = df['rating'].astype('float16')

# df_tmp = df[df['rating'] <= 3].iloc[:500]['text']

In [ ]:
df.head()

,address,name_ru,rating,rubrics,text
0,"Воронежская область, Богучарский район, М-4 До...",У тещи,4.0,Кафе,Глубинка страны во всех своих проявлениях. Асс...
1,"Москва, 4-й Кожевнический переулок, 4",Jinju,5.0,Кафе;Кофейня,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку..."
2,"Москва, 4-й Кожевнический переулок, 4",Jinju,4.0,Кафе;Кофейня,"Не очень удобное расположение, от метро идти м..."
3,"Краснодарский край, городской округ Сочи, посё...",Пандок,2.0,Ресторан,"Самый большой плюс это месторасположение, набе..."
4,"Краснодарский край, городской округ Сочи, посё...",Пандок,5.0,Ресторан,Добрый день! Сегодня во второй раз посетили с ...


In [11]:
a = [[['Первый', 'первого', 'предложения', 'реально'],
      ['Второй', 'первого', 'предложения']],
      [['первый', 'второго'],
       ['Второй', 'второго']]]
a

[[['Первый', 'первого', 'предложения', 'реально'],
  ['Второй', 'первого', 'предложения']],
 [['первый', 'второго'], ['Второй', 'второго']]]

In [15]:
ser_1 = df.iloc[0]

In [25]:
df.iloc[:4]

,address,name_ru,rating,rubrics,text
0,"Воронежская область, Богучарский район, М-4 До...",У тещи,4.0,Кафе,Глубинка страны во всех своих проявлениях. Асс...
1,"Москва, 4-й Кожевнический переулок, 4",Jinju,5.0,Кафе;Кофейня,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку..."
2,"Москва, 4-й Кожевнический переулок, 4",Jinju,4.0,Кафе;Кофейня,"Не очень удобное расположение, от метро идти м..."
3,"Краснодарский край, городской округ Сочи, посё...",Пандок,2.0,Ресторан,"Самый большой плюс это месторасположение, набе..."


In [24]:
for otsiv, my_list in zip(df.iloc(), a):
  print("новый отзыв")
  for word_x in my_list:
    series_tmp = pd.Series(word_x)
    t = otsiv.append(series_tmp)
    print(t)


новый отзыв
address    Воронежская область, Богучарский район, М-4 До...
name_ru                                               У тещи
rating                                                   4.0
rubrics                                                 Кафе
text       Глубинка страны во всех своих проявлениях. Асс...
0                                                     Первый
1                                                    первого
2                                                предложения
3                                                    реально
dtype: object
address    Воронежская область, Богучарский район, М-4 До...
name_ru                                               У тещи
rating                                                   4.0
rubrics                                                 Кафе
text       Глубинка страны во всех своих проявлениях. Асс...
0                                                     Второй
1                                                    первог

<ipython-input-26-5bd295b10691>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  t = otsiv.append(series_tmp)
<ipython-input-26-5bd295b10691>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  t = otsiv.append(series_tmp)
<ipython-input-26-5bd295b10691>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  t = otsiv.append(series_tmp)
<ipython-input-26-5bd295b10691>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  t = otsiv.append(series_tmp)


# Токенизация, стемминг, удаление стоп слов

In [2]:
# есть проблема - nчто можно улучшить - маловато места посадки. n английская остается.
tokenized = list(sent_to_words(df_tmp))
tokenized[0][:10]

NameError: ignored

In [ ]:
# стемминг и удаление стоп слов
stemmer = SnowballStemmer('russian')

stemming = []
for sentence in tokenized:
    stemming.append(list(stemmer.stem(word_x) for word_x in sentence if word_x not in stop_words))
stemming[0][:30], len(stemming[0])

(['самы',
  'большо',
  'плюс',
  'набережн',
  'шикарны',
  'вид',
  'мор',
  'красив',
  'уютн',
  'собствен',
  'плюс',
  'законч',
  'огорча',
  'отношен',
  'посетител',
  'официант',
  'неприветлив',
  'не',
  'здравству',
  'не',
  'свидан',
  'лиц',
  'недовольн',
  'неприятн',
  'не',
  'хочет',
  'смотрет',
  'так',
  'кухн',
  'оставля'],
 50)

In [ ]:
# сборка
tok_stem_text = []
for word_x in stemming:
    tok_stem_text.append(" ".join(word_x))
tok_stem_text[0]

'самы большо плюс набережн шикарны вид мор красив уютн собствен плюс законч огорча отношен посетител официант неприветлив не здравству не свидан лиц недовольн неприятн не хочет смотрет так кухн оставля жела лучш люл кебаб кост попада шашлык говядин сухо невкусны отдых позитив денег не жале хочет приход туд нам рад'

In [ ]:
# сборка с учетом не
tok_stem_text = []
for sentence_x in stemming:
    string_tmp = " ".join(sentence_x).replace("не ", 'не')
    tok_stem_text.append(string_tmp)

# заново разбиваем на токены
tokenized = list(sent_to_words(tok_stem_text))

# вывод результата
tok_stem_text[0], tokenized[0][:10]

('самы большо плюс набережн шикарны вид мор красив уютн собствен плюс законч огорча отношен посетител официант неприветлив нездравству несвидан лиц недовольн неприятн нехочет смотрет так кухн оставля жела лучш люл кебаб кост попада шашлык говядин сухо невкусны отдых позитив денег нежале хочет приход туд нам рад',
 ['самы',
  'большо',
  'плюс',
  'набережн',
  'шикарны',
  'вид',
  'мор',
  'красив',
  'уютн',
  'собствен'])

# Кодирование

## gensim doc2bow

## gensim tfidf

In [ ]:
#  сохранение извлеченных токенов в словарь
my_dictionary = corpora.Dictionary(stemming)
print(my_dictionary)

Dictionary(5122 unique tokens: ['большо', 'вид', 'говядин', 'денег', 'жале']...)


In [ ]:
# преобразование слов в Bag of Word
bow_corpus =[my_dictionary.doc2bow(doc, allow_update = True) for doc in stemming]
print(bow_corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 4), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1)], [(18, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 3), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1)], [(18, 4), (67, 2), (68, 3), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 2), (94, 1), (95, 1), (96, 1), (97, 1), (98, 3)], [(10, 1), (17, 1), (18, 3), (28, 1), (50, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1),

In [ ]:
# Вес слова в корпусе Bag of Word перед применением TF-IDF (частота слов):
word_weight =[]
for doc in bow_corpus:
    for id, freq in doc:
        word_weight.append([my_dictionary[id], freq])
print(word_weight)

[['большо', 1], ['вид', 1], ['говядин', 1], ['денег', 1], ['жале', 1], ['жела', 1], ['законч', 1], ['здравству', 1], ['кебаб', 1], ['кост', 1], ['красив', 1], ['кухн', 1], ['лиц', 1], ['лучш', 1], ['люл', 1], ['мор', 1], ['набережн', 1], ['нам', 1], ['не', 4], ['невкусны', 1], ['недовольн', 1], ['неприветлив', 1], ['неприятн', 1], ['огорча', 1], ['оставля', 1], ['отдых', 1], ['отношен', 1], ['официант', 1], ['плюс', 2], ['позитив', 1], ['попада', 1], ['посетител', 1], ['приход', 1], ['рад', 1], ['самы', 1], ['свидан', 1], ['смотрет', 1], ['собствен', 1], ['сухо', 1], ['так', 1], ['туд', 1], ['уютн', 1], ['хочет', 2], ['шашлык', 1], ['шикарны', 1], ['не', 1], ['бесплатн', 1], ['брошюр', 1], ['ваш', 1], ['долг', 2], ['доставк', 3], ['заведен', 1], ['заплат', 1], ['зуб', 1], ['которы', 1], ['мнен', 1], ['мяс', 1], ['нагар', 1], ['написа', 1], ['ниспорчен', 1], ['плева', 1], ['поднос', 1], ['прият', 1], ['рекламно', 1], ['сковородк', 1], ['факт', 1], ['хруст', 1], ['чек', 1], ['не', 4], ['

In [ ]:
# Вес слов после применения TF-IDF:
# создать модель TF-IDF
tfIdf = models.TfidfModel(bow_corpus, smartirs ='ntc')

# TF-IDF вес слова
weight_tfidf =[]
for doc in tfIdf[bow_corpus]:
    for id, freq in doc:
        weight_tfidf.append([my_dictionary[id], np.around(freq, decimals=3)])
print(weight_tfidf)

[['большо', 0.109], ['вид', 0.087], ['говядин', 0.166], ['денег', 0.142], ['жале', 0.214], ['жела', 0.121], ['законч', 0.128], ['здравству', 0.166], ['кебаб', 0.19], ['кост', 0.166], ['красив', 0.109], ['кухн', 0.067], ['лиц', 0.113], ['лучш', 0.116], ['люл', 0.176], ['мор', 0.176], ['набережн', 0.19], ['нам', 0.072], ['не', 0.025], ['невкусны', 0.19], ['недовольн', 0.152], ['неприветлив', 0.19], ['неприятн', 0.126], ['огорча', 0.19], ['оставля', 0.121], ['отдых', 0.214], ['отношен', 0.089], ['официант', 0.059], ['плюс', 0.203], ['позитив', 0.214], ['попада', 0.19], ['посетител', 0.099], ['приход', 0.091], ['рад', 0.121], ['самы', 0.142], ['свидан', 0.19], ['смотрет', 0.158], ['собствен', 0.147], ['сухо', 0.147], ['так', 0.058], ['туд', 0.095], ['уютн', 0.121], ['хочет', 0.237], ['шашлык', 0.106], ['шикарны', 0.158], ['не', 0.008], ['бесплатн', 0.201], ['брошюр', 0.259], ['ваш', 0.117], ['долг', 0.213], ['доставк', 0.391], ['заведен', 0.054], ['заплат', 0.185], ['зуб', 0.23], ['которы'

In [ ]:
# corpus - токенизированный текст
corpus = stemming
lexicon = gensim.corpora.Dictionary(corpus)
tfidf   = gensim.models.TfidfModel(bow_corpus, normalize = True)
vectors = [tfidf[lexicon.doc2bow(doc)] for doc in corpus]
vectors

[[(0, 0.10904485269368769),
  (1, 0.08688021001366633),
  (2, 0.1660847108235502),
  (3, 0.14224178028818307),
  (4, 0.21377057189428444),
  (5, 0.12061885081391144),
  (6, 0.12829456001769388),
  (7, 0.1660847108235502),
  (8, 0.18992764135891732),
  (9, 0.1660847108235502),
  (10, 0.10904485269368769),
  (11, 0.06666147867647043),
  (12, 0.11248753127197421),
  (13, 0.11631347930445823),
  (14, 0.17598042108842818),
  (15, 0.17598042108842818),
  (16, 0.18992764135891732),
  (17, 0.07236441353126433),
  (18, 0.024646451846341233),
  (19, 0.18992764135891732),
  (20, 0.152137490553061),
  (21, 0.18992764135891732),
  (22, 0.12554124474433287),
  (23, 0.18992764135891732),
  (24, 0.12061885081391144),
  (25, 0.21377057189428444),
  (26, 0.08864460073660707),
  (27, 0.059370109534197954),
  (28, 0.20339662841793146),
  (29, 0.21377057189428444),
  (30, 0.18992764135891732),
  (31, 0.09914914242880972),
  (32, 0.09147343322502725),
  (33, 0.12061885081391144),
  (34, 0.14224178028818307)

## gensim doc2vec

In [ ]:
# Для обучения модели нам нужен список целевых документов
def tagged_document(list_of_ListOfWords):
    for x, ListOfWords in enumerate(list_of_ListOfWords):
        yield doc2vec.TaggedDocument(ListOfWords, [x])

# Обновите модель

# Инициализация модели
d2v_model = doc2vec.Doc2Vec(vector_size=30, # длина вектора, которым будет представлено предложение
                            min_count=2,    # min кол-во встречания слова в прпедложении для учета
                            epochs=30,      # количество эпох
                           )
# новые данные
data_new = list(tagged_document(stem_nltk))

# расширить словарный запас
d2v_model.build_vocab(data_new)

# Обучение модели Doc2Vec
d2v_model.train(data_new, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

# Анализ выходных данных
# analyze = d2v_model.infer_vector(['Мама мыла раму'])
# analyze

doc2vec_vectorizer = np.array([d2v_model.infer_vector([text_x]) for text_x in tok_stem_text])

# Моделирование

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=vectors,         # связь id слова - его значение
                                           id2word=my_dictionary,   # результат corpora.Dictionary(stemming)
                                           num_topics=5,            # количество тем
                                           random_state=42,
                                           update_every=5,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

[(0,
  '0.000*"придупред" + 0.000*"реж" + 0.000*"нстеик" + 0.000*"нлист" + '
  '0.000*"нкурины" + 0.000*"нцен" + 0.000*"сомнительны" + 0.000*"жду" + '
  '0.000*"нохра" + 0.000*"даеш"'),
 (1,
  '0.003*"вкусн" + 0.002*"ед" + 0.002*"не" + 0.002*"заведен" + 0.002*"блюд" + '
  '0.002*"мест" + 0.002*"цен" + 0.002*"заказ" + 0.002*"прост" + '
  '0.002*"официант"'),
 (2,
  '0.000*"сбор" + 0.000*"травяно" + 0.000*"шпинат" + 0.000*"батат" + '
  '0.000*"фанат" + 0.000*"футбол" + 0.000*"примус" + 0.000*"ян" + '
  '0.000*"поеха" + 0.000*"прочл"'),
 (3,
  '0.000*"нпоищ" + 0.000*"парандж" + 0.000*"кюрдюк" + 0.000*"вар" + '
  '0.000*"сутр" + 0.000*"соминк" + 0.000*"красин" + 0.000*"наладьт" + '
  '0.000*"график" + 0.000*"капучин"'),
 (4,
  '0.000*"гопник" + 0.000*"выигра" + 0.000*"азбук" + 0.000*"промзон" + '
  '0.000*"издева" + 0.000*"жильц" + 0.000*"милашенков" + 0.000*"жител" + '
  '0.000*"фабрик" + 0.000*"удон"')]
